In [1]:
import os
import pandas as pd

In [2]:
wai_path = '/content/wai_raw/'
wai_questions = os.listdir(wai_path)
wai_questions = [p for p in wai_questions if p[0]!="."]
wai_questions

['cc2.csv',
 'cc9.csv',
 'cc15.csv',
 'cc64.csv',
 'cc34.csv',
 'cc24.csv',
 'cc60.csv',
 'cc39.csv',
 'cc30.csv',
 'cc36.csv',
 'cc16.csv',
 'cc50.csv',
 'cc6.csv',
 'cc3.csv',
 'cc25.csv',
 'cc58.csv',
 'cc22.csv',
 'cc33.csv',
 'cc46.csv',
 'cc21.csv',
 'cc54.csv',
 'cc28.csv',
 'cc17.csv',
 'cc52.csv',
 'cc29.csv',
 'cc57.csv',
 'cc53.csv',
 'cc42.csv',
 'cc27.csv',
 'cc7.csv',
 'cc62.csv',
 'cc19.csv',
 'cc48.csv',
 'cc18.csv',
 'cc38.csv',
 'cc5.csv',
 'cc41.csv',
 'cc61.csv',
 'cc12.csv',
 'cc11.csv',
 'cc45.csv',
 'cc32.csv',
 'cc23.csv',
 'cc4.csv',
 'cc14.csv',
 'cc8.csv',
 'cc63.csv',
 'cc1.csv',
 'cc43.csv',
 'cc56.csv',
 'cc31.csv',
 'cc51.csv',
 'cc47.csv',
 'cc10.csv',
 'cc55.csv',
 'cc37.csv',
 'cc20.csv',
 'cc35.csv',
 'cc44.csv',
 'cc26.csv',
 'cc40.csv',
 'cc59.csv',
 'cc49.csv',
 'cc13.csv']

# 1. 문제 별 학생 답안을 하나의 문서로 합치기

In [3]:
all_questions_answers = []
for wai_q in wai_questions :
    q_pd = pd.read_csv(os.path.join(wai_path, wai_q))
    # print(q_pd.columns)
    students_answers = list(q_pd[q_pd.columns[2]])
    # print(students_answers)
    students_answers_unique = [] # 완전 동일한 답변은 한번만 집어 넣기 
    for ans in students_answers:
        if ans==ans:
            if ans not in students_answers_unique:
                students_answers_unique.append(ans)
    anwers = " ".join(students_answers_unique)
    all_questions_answers.append((q_pd.columns[2], anwers))

for q in all_questions_answers[:10]:
    print(f"{q[0]}: ")
    print(q[1][:100])
    print("답변 총길이 : ", len(q[1]))
    print()

2. I bought a winter heating device, and I heard that it is better to install it at the bottom of the classroom, not on the wall of the classroom, to make the entire classroom warm. Let's explain in detail why it's good to install a heating device at the bottom of the classroom using our knowledge of the movement of heat in the gas. : 
Since warm air goes up and cold air goes down, it is efficient to install heating devices above and 
답변 총길이 :  60988

9. There can be clouds in the sky even on sunny days, and on cloudy days, the whole sky is full of clouds. Why do clouds form? Let's explain in detail using our knowledge of cloud generation principles. : 
I don't know. The water evaporates, becomes steam, condenses, Clouds form due to rising air currents
답변 총길이 :  58994

15. In summer, I heard thunder as lightning struck in the sky. After a little time after the lightning struck first, I heard thunder. Why do lightning come first and thunder comes later? Let's explain it in detail using 

# 2. 데이터 전처리

In [4]:
# all_questions_answers

# 전처리 
import re

## 특수 문자 제거
all_questions_answers_prep = []
for (q, ans) in all_questions_answers:
    if ans==ans:
        #print(ans)
        n_ans = re.sub(r"[^a-zA-Z가-힣\s\t\.]", "", ans)# re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣\s\t\.]", "", ans)
        all_questions_answers_prep.append((q, n_ans))

print(len(all_questions_answers_prep))


## 빈칸인 답변은 지우기
all_questions_answers_prep_2 = []
for q, ans in all_questions_answers_prep:
    if not re.sub(r"[^a-zA-Z가-힣]", "", ans):
        continue
    elif ans:
        all_questions_answers_prep_2.append((q, ans))

for i in range(50):
    print(all_questions_answers[i][0], " ",  all_questions_answers[i][1][:50])
    print(all_questions_answers_prep_2[i][0], " ", all_questions_answers_prep_2[i][1][:50])
    print()
print("전처리 후 샘플 수 : ", len(all_questions_answers_prep_2)) 

64
2. I bought a winter heating device, and I heard that it is better to install it at the bottom of the classroom, not on the wall of the classroom, to make the entire classroom warm. Let's explain in detail why it's good to install a heating device at the bottom of the classroom using our knowledge of the movement of heat in the gas.    Since warm air goes up and cold air goes down, it 
2. I bought a winter heating device, and I heard that it is better to install it at the bottom of the classroom, not on the wall of the classroom, to make the entire classroom warm. Let's explain in detail why it's good to install a heating device at the bottom of the classroom using our knowledge of the movement of heat in the gas.    Since warm air goes up and cold air goes down it i

9. There can be clouds in the sky even on sunny days, and on cloudy days, the whole sky is full of clouds. Why do clouds form? Let's explain in detail using our knowledge of cloud generation principles.    I don't know

# 데이터 Tokenize

In [11]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 461 kB/s 
     |████████████████████████████████| 453 kB 48.3 MB/s 


In [5]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [52]:
# from konlpy.tag import Mecab
# from pprint import pprint
# tokenizer= Mecab()

tokenized_ans =[]
for q, ans in all_questions_answers_prep_2:
    pos = text_to_word_sequence(ans)
    pos_cleaned = []
    for p in pos:
        # print(p)
        if p not in ['because', 'dont', 'know',"I",',']: # 조사, 부호, 어미 제거 + "모르겠다"제거(키워드 뽑마야 하므로) + "때문"도 너무 많이 나오므로 제거(질문을 구분하는 키워드는 아님)
            pos_cleaned.append(p)
    tokenized_ans.append(pos_cleaned)

In [ ]:
# print('단어 토큰화3 :',text_to_word_sequence("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))

In [53]:
# tokenized_ans

[['since',
  'warm',
  'air',
  'goes',
  'up',
  'and',
  'cold',
  'air',
  'goes',
  'down',
  'it',
  'is',
  'efficient',
  'to',
  'install',
  'heating',
  'devices',
  'above',
  'and',
  'below',
  'air',
  'conditioners',
  'convection',
  'phenomenon',
  'convection',
  'of',
  'air',
  'warm',
  'air',
  'is',
  'low',
  'in',
  'density',
  'so',
  'it',
  'goes',
  'up',
  'warm',
  'air',
  'goes',
  'up',
  'and',
  'cold',
  'air',
  'goes',
  'down',
  'so',
  'air',
  'conditioners',
  'are',
  'recommended',
  'to',
  'be',
  'installed',
  'above',
  'and',
  'hot',
  'air',
  'fans',
  'on',
  'the',
  'bottom',
  'this',
  'is',
  'warm',
  'air',
  'moves',
  'up',
  'and',
  'cold',
  'air',
  'moves',
  'down',
  'cold',
  'air',
  'sinks',
  'due',
  'to',
  'its',
  'high',
  'density',
  'and',
  'warm',
  'air',
  'floats',
  'upward',
  'due',
  'to',
  'its',
  'low',
  'density',
  'it',
  'is',
  'efficient',
  'to',
  'install',
  'a',
  'heating',
  

In [7]:
from pprint import pprint

pprint(tokenized_ans[-1][:50])

['difference',
 'in',
 'density',
 'by',
 'temperature',
 'the',
 'hot',
 'air',
 'goes',
 'up',
 'temperature',
 'difference',
 'according',
 'to',
 'temperature',
 'warm',
 'heat',
 'goes',
 'up',
 'density',
 'difference',
 'in',
 'density',
 'density',
 'difference',
 'according',
 'to',
 'temperature',
 'and',
 'the',
 'gas',
 'molecules',
 'become',
 'more',
 'active',
 'its',
 'of',
 'the',
 'difference',
 'in',
 'air',
 'density',
 'according',
 'to',
 'temperature',
 'when',
 'the',
 'air',
 'gets',
 'warm']


# 4. LDA Topic Modeling

In [8]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_ans)
corpus = [dictionary.doc2bow(text) for text in tokenized_ans]
print(corpus[1][:10]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(0, 171), (1, 1), (2, 12), (12, 3), (13, 12), (14, 244), (15, 2), (18, 1), (19, 9), (20, 519)]


In [9]:
import gensim
NUM_TOPICS = 64 # k=64=질문 개수
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)

In [10]:
for topic in topics:
    print(topic)
    words = topic[1].split('+')
    words = [re.sub(r'[\"\s]', "", w.split("*")[1]) for w in words]
    print(words)
    print()

(14, '0.076*"the" + 0.052*"and" + 0.037*"hypothesis" + 0.034*"a" + 0.030*"data" + 0.025*"inquiry" + 0.024*"to" + 0.020*"of" + 0.020*"experiment" + 0.019*"results"')
['the', 'and', 'hypothesis', 'a', 'data', 'inquiry', 'to', 'of', 'experiment', 'results']

(41, '0.081*"the" + 0.047*"will" + 0.045*"and" + 0.029*"be" + 0.022*"to" + 0.020*"of" + 0.020*"not" + 0.020*"is" + 0.017*"earth" + 0.015*"bodies"')
['the', 'will', 'and', 'be', 'to', 'of', 'not', 'is', 'earth', 'bodies']

(3, '0.002*"the" + 0.001*"and" + 0.001*"to" + 0.001*"is" + 0.001*"of" + 0.001*"a" + 0.001*"it" + 0.000*"in" + 0.000*"be" + 0.000*"water"')
['the', 'and', 'to', 'is', 'of', 'a', 'it', 'in', 'be', 'water']

(59, '0.001*"the" + 0.000*"to" + 0.000*"and" + 0.000*"of" + 0.000*"water" + 0.000*"it" + 0.000*"a" + 0.000*"is" + 0.000*"in" + 0.000*"you"')
['the', 'to', 'and', 'of', 'water', 'it', 'a', 'is', 'in', 'you']

(42, '0.001*"the" + 0.001*"and" + 0.001*"to" + 0.000*"of" + 0.000*"is" + 0.000*"it" + 0.000*"a" + 0.000*"in" 

In [11]:
for wai_q in wai_questions :
    q_pd = pd.read_csv(os.path.join(wai_path, wai_q))
    cols = q_pd.columns[2:]
    # print(cols)
    cols = [c for c in cols if "Unnamed" not in c]
    # print(cols)
    print(wai_q.split(".")[0])
    print(cols[0])  # 문제만 출력
    kwds = [re.sub("q[0-9]+_", "", c) for c in cols[1:]]
    # print(kwds)
    keywords = []
    for k in kwds:
        k = re.sub(".[0-9]", "", k)
        if k  not in keywords:
            keywords.append(k)   
    print(keywords)
    print()

cc2
2. I bought a winter heating device, and I heard that it is better to install it at the bottom of the classroom, not on the wall of the classroom, to make the entire classroom warm. Let's explain in detail why it's good to install a heating device at the bottom of the classroom using our knowledge of the movement of heat in the gas. 
[',convection,density difference,convection,density difference']

cc9
9. There can be clouds in the sky even on sunny days, and on cloudy days, the whole sky is full of clouds. Why do clouds form? Let's explain in detail using our knowledge of cloud generation principles. 
[', evaporation, insulation expansion, condensation, airflow, evaporation,insulation expansion,condensin.airflow']

cc15
15. In summer, I heard thunder as lightning struck in the sky. After a little time after the lightning struck first, I heard thunder. Why do lightning come first and thunder comes later? Let's explain it in detail using our knowledge of light and sound.
[', speed d

# 5. 명사, 형용사만 살리기 & 다시 LDA Topic Modeling

In [54]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [55]:
import nltk

In [56]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [58]:
tokenized_ans =[]
for q, ans in all_questions_answers_prep_2:
    pos = word_tokenize(ans)
    pos = pos_tag(pos)
    pos_cleaned = []
    for p in pos:
        # print(p)
        if p[1] in ['NNPS', 'NNP', 'VB','VBD', 'NN', 'NNS'] and p not in ['because', 'dont', 'know',"I",',']: # 조사, 부호, 어미 제거 + "모르겠다"제거(키워드 뽑마야 하므로) + "때문"도 너무 많이 나오므로 제거(질문을 구분하는 키워드는 아님)
            pos_cleaned.append(p)
    tokenized_ans.append(pos_cleaned)

In [35]:
tokenized_ans[0][0]

('air', 'NN')

In [18]:
len(tokenized_ans)

64

In [86]:
tokenized_ans_list1 = []
for i in range(0,64):
    tokenized_ans_list = []
    tokenized_ans_list.append(list(tokenized_ans[0][i]))
    # print(tokenized_ans_list)
    # print(tokenized_ans_list[0][1])
    del tokenized_ans_list[0][1]
    # print(tokenized_ans_list)
    # print(tokenized_ans_list[0][0])
    # pd.Series(tokenized_ans_list)
    # print(tokenized_ans_list)
    tokenized_ans_list1.append(tokenized_ans_list[0][0])
    # tokenized_ans_list1 = pd.Series(tokenized_ans_list1).unique()
# print(tokenized_ans_list1)
list(tokenized_ans_list1)
tokenized_ans_list2 = []
tokenized_ans_list2.append(tokenized_ans_list1)
print(tokenized_ans_list2)

[['air', 'air', 'install', 'devices', 'air', 'conditioners', 'convection', 'phenomenon', 'convection', 'air', 'Warm', 'air', 'density', 'Warm', 'air', 'air', 'conditioners', 'be', 'air', 'fans', 'bottom', 'air', 'cold', 'air', 'moves', 'Cold', 'air', 'sinks', 'density', 'air', 'floats', 'density', 'install', 'heating', 'device', 'bottom', 'air', 'be', 'circulate', 'air', 'Warm', 'air', 'air', 'air', 'air', 'air', 'air', 'density', 'wind', 'be', 'air', 'Hot', 'air', 'characteristic', 'bottom', 'bottom', 'install', 'air', 'sinks', 'convection', 'phenomenon', 'gas', 'install', 'heating']]


In [62]:
# tokenized_ans_list1
from pprint import pprint

pprint(tokenized_ans_list2[-1][:50])

['air',
 'air',
 'install',
 'devices',
 'air',
 'conditioners',
 'convection',
 'phenomenon',
 'convection',
 'air',
 'Warm',
 'air',
 'density',
 'Warm',
 'air',
 'air',
 'conditioners',
 'be',
 'air',
 'fans',
 'bottom',
 'air',
 'cold',
 'air',
 'moves',
 'Cold',
 'air',
 'sinks',
 'density',
 'air',
 'floats',
 'density',
 'install',
 'heating',
 'device',
 'bottom',
 'air',
 'be',
 'circulate',
 'air',
 'Warm',
 'air',
 'air',
 'air',
 'air',
 'air',
 'air',
 'density',
 'wind',
 'be']


In [64]:
# tokenized_ans_list1

In [66]:
# tokenized_ans_list1[0]

In [67]:
# def listToString(str_list):
#     result = ""
#     for s in str_list:
#         for j in s:
#             result += s + " "
#         return result.strip()

In [68]:
# str_list = tokenized_ans_list1[0]
# result = listToString(str_list)
# print(result)

In [69]:
# list(tokenized_ans_list)
# print(tokenized_ans_list)

In [74]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_ans_list2)
corpus = [dictionary.doc2bow(text) for text in tokenized_ans_list2]
# print(corpus[0])
print(corpus[0][:10]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(0, 1), (1, 1), (2, 3), (3, 23), (4, 3), (5, 4), (6, 1), (7, 1), (8, 1), (9, 2)]


In [75]:
import gensim
NUM_TOPICS = 64 # k=64=질문 개수
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)

In [78]:
for topic in topics:
    print(topic)
    words = topic[1].split('+')
    words = [re.sub(r'[\"\s]', "", w.split("*")[1]) for w in words]
    print(words)
    print()

(16, '0.043*"heating" + 0.043*"install" + 0.043*"fans" + 0.043*"floats" + 0.043*"gas" + 0.043*"device" + 0.043*"sinks" + 0.043*"phenomenon" + 0.043*"density" + 0.043*"moves"')
['heating', 'install', 'fans', 'floats', 'gas', 'device', 'sinks', 'phenomenon', 'density', 'moves']

(24, '0.043*"heating" + 0.043*"install" + 0.043*"fans" + 0.043*"floats" + 0.043*"gas" + 0.043*"device" + 0.043*"sinks" + 0.043*"phenomenon" + 0.043*"density" + 0.043*"moves"')
['heating', 'install', 'fans', 'floats', 'gas', 'device', 'sinks', 'phenomenon', 'density', 'moves']

(17, '0.043*"heating" + 0.043*"install" + 0.043*"fans" + 0.043*"floats" + 0.043*"gas" + 0.043*"device" + 0.043*"sinks" + 0.043*"phenomenon" + 0.043*"density" + 0.043*"moves"')
['heating', 'install', 'fans', 'floats', 'gas', 'device', 'sinks', 'phenomenon', 'density', 'moves']

(61, '0.043*"heating" + 0.043*"install" + 0.043*"fans" + 0.043*"floats" + 0.043*"gas" + 0.043*"device" + 0.043*"sinks" + 0.043*"phenomenon" + 0.043*"density" + 0.043*

In [79]:
for wai_q in wai_questions :
    q_pd = pd.read_csv(os.path.join(wai_path, wai_q))
    cols = q_pd.columns[1:]
    cols = [c for c in cols if "Unnamed" not in c]
    print(wai_q.split(".")[0])
    print(cols[0])
    kwds = [re.sub("q[0-9]+_", "", c) for c in cols[1:]]
    keywords = []
    for k in kwds:
        k_ = re.sub(".[0-9]", "", k)
        if k_  not in keywords:
            keywords.append(k_)   
    print(keywords)
    print()

cc2
2. I bought a winter heating device, and I heard that it is better to install it at the bottom of the classroom, not on the wall of the classroom, to make the entire classroom warm. Let's explain in detail why it's good to install a heating device at the bottom of the classroom using our knowledge of the movement of heat in the gas. 
[',convection,density difference,convection,density difference']

cc9
9. There can be clouds in the sky even on sunny days, and on cloudy days, the whole sky is full of clouds. Why do clouds form? Let's explain in detail using our knowledge of cloud generation principles. 
[', evaporation, insulation expansion, condensation, airflow, evaporation,insulation expansion,condensin.airflow']

cc15
15. In summer, I heard thunder as lightning struck in the sky. After a little time after the lightning struck first, I heard thunder. Why do lightning come first and thunder comes later? Let's explain it in detail using our knowledge of light and sound.
[', speed d

In [88]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 30.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=2b7fb8d5c1ed18031b3a5a7b9b19acc547cffb70806463f6787016a42fd37bbb
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2042475e2bfbe31384d4223ecb89e5dcec18efe46cc4c8ed7358f578cb1b70d6
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [90]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary, mds='mmds')
pyLDAvis.display(vis)